## Capstone 2: Data Wrangling


### Overview

How can we use data from 2015-2018 to predict/assess what type of pitch should be thrown in a given at bat?

This can be looked at from both the pitching team’s perspective (what is the ideal pitch for a given situation), and from the batting team (what pitch should I expect, assuming that the pitcher will choose the optimal pitch). Using the ab_id, we can link the data in the pitches csv to the data in the atbats csv to look at the outcomes of at bats along with the exact type and order of pitches thrown. We can add to this by using the batter_id and pitcher_id to gather specific data for a given hitter or pitcher by linking with the player_names csv. 

The data will be acquired from https://www.kaggle.com/pschale/mlb-pitch-data-20152018?select=pitches.csv
which was scraped from  http://gd2.mlb.com/components/game/mlb/.


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pandas_profiling
from library.sb_utils import save_file


In [2]:
from pathlib import Path
import pandas_profiling
import requests
from pandas_profiling.utils.cache import cache_file

In [3]:
import datetime as dt
from datetime import datetime

### Because I found myself repeatedly using this technique, I decided to define the following function to show missing values

In [4]:
#missing values function
def missing(df):
    '''Takes dataframe as argument, 
    returns count and % of missing values for each column'''
    import pandas as pd
    miss = pd.concat([df.isnull().sum(), 100 * df.isnull().mean()], axis = 1)
    miss.columns = ['count','%']
    return miss.sort_values(by = ['count'], ascending = False)

### Import the data
#### First we will do a small file (player_names), and then do the rest

In [5]:
#the CSV data files are in the data/raw directory
#player names
names = pd.read_csv('../data/raw/player_names.csv')

In [6]:
names.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2218 entries, 0 to 2217
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          2218 non-null   int64 
 1   first_name  2218 non-null   object
 2   last_name   2218 non-null   object
dtypes: int64(1), object(2)
memory usage: 52.1+ KB


In [7]:
names.head()

,id,first_name,last_name
0,452657,Jon,Lester
1,425794,Adam,Wainwright
2,457435,Phil,Coke
3,435400,Jason,Motte
4,519166,Neil,Ramirez


## Import Games and At-Bats Data

In [8]:
#the CSV data files are in the data/raw directory
#at bat info
atbats = pd.read_csv('../data/raw/atbats.csv')
#games info 
games = pd.read_csv('../data/raw/games.csv')


In [9]:
atbats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 740389 entries, 0 to 740388
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   ab_id       740389 non-null  int64 
 1   batter_id   740389 non-null  int64 
 2   event       740389 non-null  object
 3   g_id        740389 non-null  int64 
 4   inning      740389 non-null  int64 
 5   o           740389 non-null  int64 
 6   p_score     740389 non-null  int64 
 7   p_throws    740389 non-null  object
 8   pitcher_id  740389 non-null  int64 
 9   stand       740389 non-null  object
 10  top         740389 non-null  bool  
dtypes: bool(1), int64(7), object(3)
memory usage: 57.2+ MB


In [10]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9718 entries, 0 to 9717
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   attendance        9718 non-null   int64 
 1   away_final_score  9718 non-null   int64 
 2   away_team         9718 non-null   object
 3   date              9718 non-null   object
 4   elapsed_time      9718 non-null   int64 
 5   g_id              9718 non-null   int64 
 6   home_final_score  9718 non-null   int64 
 7   home_team         9718 non-null   object
 8   start_time        9718 non-null   object
 9   umpire_1B         9718 non-null   object
 10  umpire_2B         9715 non-null   object
 11  umpire_3B         9718 non-null   object
 12  umpire_HP         9718 non-null   object
 13  venue_name        9718 non-null   object
 14  weather           9718 non-null   object
 15  wind              9718 non-null   object
 16  delay             9718 non-null   int64 
dtypes: int64(6), o

## Import Pitch Data (800 mb)

In [11]:
#the CSV data files are in the data/raw directory
pitches = pd.read_csv('../data/raw/pitches.csv')

In [12]:
pitches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2867154 entries, 0 to 2867153
Data columns (total 40 columns):
 #   Column           Dtype  
---  ------           -----  
 0   px               float64
 1   pz               float64
 2   start_speed      float64
 3   end_speed        float64
 4   spin_rate        float64
 5   spin_dir         float64
 6   break_angle      float64
 7   break_length     float64
 8   break_y          float64
 9   ax               float64
 10  ay               float64
 11  az               float64
 12  sz_bot           float64
 13  sz_top           float64
 14  type_confidence  float64
 15  vx0              float64
 16  vy0              float64
 17  vz0              float64
 18  x                float64
 19  x0               float64
 20  y                float64
 21  y0               float64
 22  z0               float64
 23  pfx_x            float64
 24  pfx_z            float64
 25  nasty            float64
 26  zone             float64
 27  code        

In [13]:
pitches.head()

,px,pz,start_speed,end_speed,spin_rate,spin_dir,break_angle,break_length,break_y,ax,...,event_num,b_score,ab_id,b_count,s_count,outs,pitch_num,on_1b,on_2b,on_3b
0,0.416,2.963,92.9,84.1,2305.052,159.235,-25.0,3.2,23.7,7.665,...,3,0.0,2.015000e+09,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,-0.191,2.347,92.8,84.1,2689.935,151.402,-40.7,3.4,23.7,12.043,...,4,0.0,2.015000e+09,0.0,1.0,0.0,2.0,0.0,0.0,0.0
2,-0.518,3.284,94.1,85.2,2647.972,145.125,-43.7,3.7,23.7,14.368,...,5,0.0,2.015000e+09,0.0,2.0,0.0,3.0,0.0,0.0,0.0
3,-0.641,1.221,91.0,84.0,1289.590,169.751,-1.3,5.0,23.8,2.104,...,6,0.0,2.015000e+09,0.0,2.0,0.0,4.0,0.0,0.0,0.0
4,-1.821,2.083,75.4,69.6,1374.569,280.671,18.4,12.0,23.8,-10.280,...,7,0.0,2.015000e+09,1.0,2.0,0.0,5.0,0.0,0.0,0.0


## Check missing values for at bats, players, pitches

In [14]:
#missing values for each column in pitches
missingP = pd.concat([pitches.isnull().sum(), 100 * pitches.isnull().mean()], axis=1)
missingP.columns=['count', '%']
missingP.sort_values(by=['count'], ascending = False)

,count,%
px,14189,0.494881
type_confidence,14189,0.494881
pitch_type,14189,0.494881
zone,14189,0.494881
nasty,14189,0.494881
z0,14189,0.494881
y0,14189,0.494881
pz,14189,0.494881
x0,14189,0.494881
vz0,14189,0.494881


In [15]:
#missing values for each column in atbats
missingAB = pd.concat([atbats.isnull().sum(), 100 * atbats.isnull().mean()], axis=1)
missingAB.columns=['count', '%']
missingAB.sort_values(by=['count'], ascending = False)

,count,%
ab_id,0,0.0
batter_id,0,0.0
event,0,0.0
g_id,0,0.0
inning,0,0.0
o,0,0.0
p_score,0,0.0
p_throws,0,0.0
pitcher_id,0,0.0
stand,0,0.0


In [16]:
#missing values for players

missingPlayers = pd.concat([names.isnull().sum(), 100 * names.isnull().mean()], axis=1)
missingPlayers.columns=['count', '%']
missingPlayers.sort_values(by=['count'], ascending = False)

,count,%
id,0,0.0
first_name,0,0.0
last_name,0,0.0


In [17]:
#missing values for games
missingG = pd.concat([games.isnull().sum(), 100 * games.isnull().mean()], axis=1)
missingG.columns=['count', '%']
missingG.sort_values(by=['count'], ascending = False)

,count,%
umpire_2B,3,0.030871
attendance,0,0.000000
umpire_1B,0,0.000000
wind,0,0.000000
weather,0,0.000000
venue_name,0,0.000000
umpire_HP,0,0.000000
umpire_3B,0,0.000000
start_time,0,0.000000
away_final_score,0,0.000000


### There are no missing values in the AB and Names dataframes

### Because there are no columns with more than 0.5% of values missing (i.e. no large chunks missing) in the pitches and games dataframes, I am just going to remove rows with missing values.

### I think I want to concat the df's first (link the abs, pitches, and games) before removing though. 

In [18]:
#maybe do groupby ab id for pitches first. 
pitches.ab_id.unique().shape

(740241,)

In [19]:
atbats.ab_id.unique().shape

(740389,)

### It appears that there are 159 at bats missing from the pitches df. Ill start by trying to groupy abid on the pitches df and then decide how I want to join the dfs


Pitch Type Definitions
CH - Changeup

CU - Curveball

EP - Eephus*

FC - Cutter

FF - Four-seam Fastball

FO - Pitchout (also PO)*

FS - Splitter

FT - Two-seam Fastball

IN - Intentional ball

KC - Knuckle curve

KN - Knuckeball

PO - Pitchout (also FO)*

SC - Screwball*

SI - Sinker

SL - Slider

UN - Unknown*

these pitch types occur rarely
Code Definitions #
While these aren't spelled out anywhere, play descriptions allowed confident identification of these codes

B - Ball

*B - Ball in dirt

S - Swinging Strike

C - Called Strike

F - Foul

T - Foul Tip

L - Foul Bunt

I - Intentional Ball

W - Swinging Strike (Blocked)

M - Missed Bunt

P - Pitchout

Q - Swinging pitchout

R - Foul pitchout

Values that only occur on last pitch of at-bat:

X - In play, out(s)

D - In play, no out

E - In play, runs

H - Hit by pitch

Note: all codes, except for H, come directly from the XML files. All at-bats with code H were given no code in the XMLs.

In [20]:
pitches.columns; atbats.columns


Index(['ab_id', 'batter_id', 'event', 'g_id', 'inning', 'o', 'p_score',
       'p_throws', 'pitcher_id', 'stand', 'top'],
      dtype='object')

In [21]:
#at bats with names
names.head()

,id,first_name,last_name
0,452657,Jon,Lester
1,425794,Adam,Wainwright
2,457435,Phil,Coke
3,435400,Jason,Motte
4,519166,Neil,Ramirez


In [22]:
atbats.head()

,ab_id,batter_id,event,g_id,inning,o,p_score,p_throws,pitcher_id,stand,top
0,2015000001,572761,Groundout,201500001,1,1,0,L,452657,L,True
1,2015000002,518792,Double,201500001,1,1,0,L,452657,L,True
2,2015000003,407812,Single,201500001,1,1,0,L,452657,R,True
3,2015000004,425509,Strikeout,201500001,1,2,0,L,452657,R,True
4,2015000005,571431,Strikeout,201500001,1,3,0,L,452657,L,True


In [23]:
atbatNames = names.merge(atbats, how = 'right', left_on = 'id', right_on = 'pitcher_id' )

In [24]:
atbatNames.head()

,id,first_name,last_name,ab_id,batter_id,event,g_id,inning,o,p_score,p_throws,pitcher_id,stand,top
0,452657,Jon,Lester,2015000001,572761,Groundout,201500001,1,1,0,L,452657,L,True
1,452657,Jon,Lester,2015000002,518792,Double,201500001,1,1,0,L,452657,L,True
2,452657,Jon,Lester,2015000003,407812,Single,201500001,1,1,0,L,452657,R,True
3,452657,Jon,Lester,2015000004,425509,Strikeout,201500001,1,2,0,L,452657,R,True
4,452657,Jon,Lester,2015000005,571431,Strikeout,201500001,1,3,0,L,452657,L,True


In [25]:
missingTest = pd.concat([atbatNames.isnull().sum(), 100 * atbatNames.isnull().mean()], axis=1)
missingTest.columns=['count', '%']
missingTest.sort_values(by=['count'], ascending = False)

,count,%
id,0,0.0
first_name,0,0.0
last_name,0,0.0
ab_id,0,0.0
batter_id,0,0.0
event,0,0.0
g_id,0,0.0
inning,0,0.0
o,0,0.0
p_score,0,0.0


In [26]:
names[names['first_name']=='Clayton']

,id,first_name,last_name
85,477132,Clayton,Kershaw
619,453385,Clayton,Richard


### Start with Clayton Kershaw, id 477132

In [27]:
KershawAB = atbatNames[atbatNames['id']==477132]

In [28]:
KershawAB.head()

,id,first_name,last_name,ab_id,batter_id,event,g_id,inning,o,p_score,p_throws,pitcher_id,stand,top
767,477132,Clayton,Kershaw,2015000768,571976,Hit By Pitch,201500012,1,0,0,L,477132,R,True
768,477132,Clayton,Kershaw,2015000769,519083,Strikeout,201500012,1,1,0,L,477132,R,True
769,477132,Clayton,Kershaw,2015000770,461314,Single,201500012,1,1,0,L,477132,R,True
770,477132,Clayton,Kershaw,2015000771,457708,Strikeout,201500012,1,2,0,L,477132,R,True
771,477132,Clayton,Kershaw,2015000772,519025,Forceout,201500012,1,3,0,L,477132,R,True


### Not using Games for now

In [ ]:
#KershawAB.merge(games, on = 'g_id', how = 'left' )

In [29]:
#df1.merge(df2, left_on='CID', right_on='ID', how='outer')
KABp = KershawAB.merge(pitches, on = 'ab_id', how = 'outer')

In [30]:
KABp.head()

,id,first_name,last_name,ab_id,batter_id,event,g_id,inning,o,p_score,...,pitch_type,event_num,b_score,b_count,s_count,outs,pitch_num,on_1b,on_2b,on_3b
0,477132.0,Clayton,Kershaw,2.015001e+09,571976.0,Hit By Pitch,201500012.0,1.0,0.0,0.0,...,FF,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,477132.0,Clayton,Kershaw,2.015001e+09,571976.0,Hit By Pitch,201500012.0,1.0,0.0,0.0,...,FF,4.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0
2,477132.0,Clayton,Kershaw,2.015001e+09,571976.0,Hit By Pitch,201500012.0,1.0,0.0,0.0,...,FF,5.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0
3,477132.0,Clayton,Kershaw,2.015001e+09,519083.0,Strikeout,201500012.0,1.0,1.0,0.0,...,FF,8.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4,477132.0,Clayton,Kershaw,2.015001e+09,519083.0,Strikeout,201500012.0,1.0,1.0,0.0,...,FF,9.0,0.0,1.0,0.0,0.0,2.0,1.0,0.0,0.0


In [32]:
test = KABp.groupby('ab_id')
test.head()

,id,first_name,last_name,ab_id,batter_id,event,g_id,inning,o,p_score,...,pitch_type,event_num,b_score,b_count,s_count,outs,pitch_num,on_1b,on_2b,on_3b
0,477132.0,Clayton,Kershaw,2.015001e+09,571976.0,Hit By Pitch,201500012.0,1.0,0.0,0.0,...,FF,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,477132.0,Clayton,Kershaw,2.015001e+09,571976.0,Hit By Pitch,201500012.0,1.0,0.0,0.0,...,FF,4.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0
2,477132.0,Clayton,Kershaw,2.015001e+09,571976.0,Hit By Pitch,201500012.0,1.0,0.0,0.0,...,FF,5.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0
3,477132.0,Clayton,Kershaw,2.015001e+09,519083.0,Strikeout,201500012.0,1.0,1.0,0.0,...,FF,8.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4,477132.0,Clayton,Kershaw,2.015001e+09,519083.0,Strikeout,201500012.0,1.0,1.0,0.0,...,FF,9.0,0.0,1.0,0.0,0.0,2.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2867146,NaN,NaN,NaN,2.018186e+09,NaN,NaN,NaN,NaN,NaN,NaN,...,FF,564.0,1.0,3.0,1.0,2.0,5.0,0.0,0.0,0.0
2867151,NaN,NaN,NaN,2.018186e+09,NaN,NaN,NaN,NaN,NaN,NaN,...,FF,572.0,1.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0
2867152,NaN,NaN,NaN,2.018186e+09,NaN,NaN,NaN,NaN,NaN,NaN,...,FF,577.0,1.0,1.0,0.0,2.0,2.0,0.0,1.0,0.0
2867153,NaN,NaN,NaN,2.018186e+09,NaN,NaN,NaN,NaN,NaN,NaN,...,FF,578.0,1.0,1.0,1.0,2.0,3.0,0.0,1.0,0.0


In [28]:
KershawPitches = pitches[pitches['ab_id'].isin(KershawAB['ab_id'])]

In [29]:
KershawPitches.head()

,px,pz,start_speed,end_speed,spin_rate,spin_dir,break_angle,break_length,break_y,ax,...,event_num,b_score,ab_id,b_count,s_count,outs,pitch_num,on_1b,on_2b,on_3b
2895,-0.762,2.393,93.1,84.8,2112.807,176.674,1.5,3.1,23.7,1.151,...,3,0.0,2.015001e+09,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2896,0.364,2.336,93.5,86.4,2506.791,172.826,-11.8,2.3,23.8,2.953,...,4,0.0,2.015001e+09,0.0,1.0,0.0,2.0,0.0,0.0,0.0
2897,-2.070,1.936,94.2,86.8,2161.998,171.800,-3.4,2.9,23.8,2.925,...,5,0.0,2.015001e+09,0.0,2.0,0.0,3.0,0.0,0.0,0.0
2898,0.240,3.796,92.9,85.2,2142.513,168.788,-13.2,3.0,23.8,3.909,...,8,0.0,2.015001e+09,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2899,-0.093,3.031,93.0,85.5,2168.928,180.974,4.8,2.9,23.8,-0.347,...,9,0.0,2.015001e+09,1.0,0.0,0.0,2.0,1.0,0.0,0.0


In [30]:
KershawPitches.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10339 entries, 2895 to 2859929
Data columns (total 40 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   px               10253 non-null  float64
 1   pz               10253 non-null  float64
 2   start_speed      10253 non-null  float64
 3   end_speed        10253 non-null  float64
 4   spin_rate        10253 non-null  float64
 5   spin_dir         10253 non-null  float64
 6   break_angle      10253 non-null  float64
 7   break_length     10253 non-null  float64
 8   break_y          10253 non-null  float64
 9   ax               10253 non-null  float64
 10  ay               10253 non-null  float64
 11  az               10253 non-null  float64
 12  sz_bot           10333 non-null  float64
 13  sz_top           10333 non-null  float64
 14  type_confidence  10253 non-null  float64
 15  vx0              10253 non-null  float64
 16  vy0              10253 non-null  float64
 17  vz0    

In [31]:
Kershaw = KershawAB.merge(KershawPitches, how = 'right', left_on = 'ab_id', right_on = 'ab_id' )

In [32]:
Kershaw.head()

,id,first_name,last_name,ab_id,batter_id,event,g_id,inning,o,p_score,...,pitch_type,event_num,b_score,b_count,s_count,outs,pitch_num,on_1b,on_2b,on_3b
0,477132,Clayton,Kershaw,2015000768,571976,Hit By Pitch,201500012,1,0,0,...,FF,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,477132,Clayton,Kershaw,2015000768,571976,Hit By Pitch,201500012,1,0,0,...,FF,4,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0
2,477132,Clayton,Kershaw,2015000768,571976,Hit By Pitch,201500012,1,0,0,...,FF,5,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0
3,477132,Clayton,Kershaw,2015000769,519083,Strikeout,201500012,1,1,0,...,FF,8,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4,477132,Clayton,Kershaw,2015000769,519083,Strikeout,201500012,1,1,0,...,FF,9,0.0,1.0,0.0,0.0,2.0,1.0,0.0,0.0


In [33]:
Kershaw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10339 entries, 0 to 10338
Data columns (total 53 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               10339 non-null  int64  
 1   first_name       10339 non-null  object 
 2   last_name        10339 non-null  object 
 3   ab_id            10339 non-null  int64  
 4   batter_id        10339 non-null  int64  
 5   event            10339 non-null  object 
 6   g_id             10339 non-null  int64  
 7   inning           10339 non-null  int64  
 8   o                10339 non-null  int64  
 9   p_score          10339 non-null  int64  
 10  p_throws         10339 non-null  object 
 11  pitcher_id       10339 non-null  int64  
 12  stand            10339 non-null  object 
 13  top              10339 non-null  bool   
 14  px               10253 non-null  float64
 15  pz               10253 non-null  float64
 16  start_speed      10253 non-null  float64
 17  end_speed   

In [34]:
Kershaw.describe()

,id,ab_id,batter_id,g_id,inning,o,p_score,pitcher_id,px,pz,...,zone,event_num,b_score,b_count,s_count,outs,pitch_num,on_1b,on_2b,on_3b
count,10339.0,1.033900e+04,10339.000000,1.033900e+04,10339.000000,10339.000000,10339.000000,10339.0,10253.000000,10253.000000,...,10253.000000,10339.000000,10339.000000,10339.000000,10339.000000,10339.000000,10339.000000,10339.000000,10339.000000,10339.000000
mean,477132.0,2.016458e+09,516826.988393,2.016384e+08,3.974949,1.761873,1.740014,477132.0,-0.153484,2.144890,...,9.470204,221.030467,0.777348,0.767773,0.933553,0.983267,2.821549,0.238805,0.137828,0.064126
std,0.0,1.164462e+06,73940.010607,1.161065e+05,2.108594,0.921977,2.087447,0.0,0.684468,1.031770,...,4.164466,135.175598,1.080949,0.908539,0.827871,0.814843,1.678035,0.426374,0.344736,0.244989
min,477132.0,2.015001e+09,112526.000000,2.015000e+08,1.000000,0.000000,0.000000,477132.0,-2.651000,-2.693544,...,1.000000,3.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,477132.0,2.015139e+09,456488.000000,2.015018e+08,2.000000,1.000000,0.000000,477132.0,-0.619000,1.492241,...,6.000000,106.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
50%,477132.0,2.016166e+09,518452.000000,2.016022e+08,4.000000,2.000000,1.000000,477132.0,-0.193195,2.192000,...,11.000000,213.000000,0.000000,1.000000,1.000000,1.000000,2.000000,0.000000,0.000000,0.000000
75%,477132.0,2.017172e+09,572761.000000,2.017022e+08,6.000000,3.000000,3.000000,477132.0,0.269000,2.835000,...,13.000000,328.000000,1.000000,1.000000,2.000000,2.000000,4.000000,0.000000,0.000000,0.000000
max,477132.0,2.018184e+09,670950.000000,2.018024e+08,9.000000,3.000000,12.000000,477132.0,6.286000,6.818188,...,14.000000,562.000000,5.000000,3.000000,2.000000,2.000000,12.000000,1.000000,1.000000,1.000000


### Now include game ID

In [35]:
CKdata = Kershaw.merge(games, on = 'g_id', how = 'left' )

In [36]:
CKdata.columns


Index(['id', 'first_name', 'last_name', 'ab_id', 'batter_id', 'event', 'g_id',
       'inning', 'o', 'p_score', 'p_throws', 'pitcher_id', 'stand', 'top',
       'px', 'pz', 'start_speed', 'end_speed', 'spin_rate', 'spin_dir',
       'break_angle', 'break_length', 'break_y', 'ax', 'ay', 'az', 'sz_bot',
       'sz_top', 'type_confidence', 'vx0', 'vy0', 'vz0', 'x', 'x0', 'y', 'y0',
       'z0', 'pfx_x', 'pfx_z', 'nasty', 'zone', 'code', 'type', 'pitch_type',
       'event_num', 'b_score', 'b_count', 's_count', 'outs', 'pitch_num',
       'on_1b', 'on_2b', 'on_3b', 'attendance', 'away_final_score',
       'away_team', 'date', 'elapsed_time', 'home_final_score', 'home_team',
       'start_time', 'umpire_1B', 'umpire_2B', 'umpire_3B', 'umpire_HP',
       'venue_name', 'weather', 'wind', 'delay'],
      dtype='object')

In [37]:
#missing values for each column
missingCK = pd.concat([CKdata.isnull().sum(), 100 * CKdata.isnull().mean()], axis=1)
missingCK.columns=['count', '%']
missingCK.sort_values(by=['count'], ascending = False).head(len(CKdata.columns))

,count,%
ax,86,0.831802
pz,86,0.831802
vy0,86,0.831802
vx0,86,0.831802
type_confidence,86,0.831802
...,...,...
code,0,0.000000
type,0,0.000000
event_num,0,0.000000
b_score,0,0.000000


    #missing values function
    def missing(df):
        '''Takes dataframe as argument, 
        returns count and % of missing values for each column'''
        import pandas as pd
        miss = pd.concat([df.isnull().sum(), 100 * df.isnull().mean()], axis = 1)
        miss.columns = ['count','%']
        return miss.sort_values(by = ['count'], ascending = False)

In [38]:
missing(CKdata)

,count,%
ax,86,0.831802
pz,86,0.831802
vy0,86,0.831802
vx0,86,0.831802
type_confidence,86,0.831802
...,...,...
code,0,0.000000
type,0,0.000000
event_num,0,0.000000
b_score,0,0.000000


    report = df.profile_report(sort=None, html={'style':{'full_width': True}}, progress_bar=False)
    report
    profile_report.to_file("tmp/example.html")
    profile = ProfileReport(df, minimal=True)
    
    profile.to_file(output_file="output_min.html")

In [39]:
CKdata['date'].head(10)

0    2015-04-06
1    2015-04-06
2    2015-04-06
3    2015-04-06
4    2015-04-06
5    2015-04-06
6    2015-04-06
7    2015-04-06
8    2015-04-06
9    2015-04-06
Name: date, dtype: object

In [43]:
CKdata['date'] = pd.to_datetime(CKdata['date'], infer_datetime_format=True)

In [45]:
CKdata.dtypes

id             int64
first_name    object
last_name     object
ab_id          int64
batter_id      int64
               ...  
umpire_HP     object
venue_name    object
weather       object
wind          object
delay          int64
Length: 69, dtype: object

In [48]:
kersh2015 = CKdata[CKdata.date < datetime.fromisoformat('2016-01-01')]

In [49]:
kersh2015.shape

(3392, 69)

In [ ]:
results2015 = kersh2015.groupby('ab_id')

In [ ]:
#https://stackoverflow.com/questions/46140609/one-to-many-left-outer-join-with-pandas-python